# Quantum codes do not fix qubit independent errors
#### Authors: J. Lacalle, L. M. Pozo-Coronado, A. L. Fonseca de Oliveira, R. Martin-Cuevas

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## 1. Finding the basis associated with the 5-qubit quantum code

First, we will import the required libraries and set up a few basic variables.

In [2]:
from sympy import symbols, Symbol, Matrix, I, pprint, diff, re, im
from sympy.core.expr import Expr
from sympy.physics.quantum import Dagger
from sympy.physics.quantum.tensorproduct import TensorProduct

num_qubits = 5

Then, we define the generators used by the 5-qubit quantum code $C$. As we will be using the string themselves to perform calculations on the quantum states, the Dirac notation is avoided when defining these variables.

In [3]:
L0 = Symbol('00000') - Symbol('00011') + Symbol('00101') - Symbol('00110') + \
     Symbol('01001') + Symbol('01010') - Symbol('01100') - Symbol('01111') - \
     Symbol('10001') + Symbol('10010') + Symbol('10100') - Symbol('10111') - \
     Symbol('11000') - Symbol('11011') - Symbol('11101') - Symbol('11110')

L1 = - Symbol('00001') - Symbol('00010') - Symbol('00100') - Symbol('00111') - \
     Symbol('01000') + Symbol('01011') + Symbol('01101') - Symbol('01110') - \
     Symbol('10000') - Symbol('10011') + Symbol('10101') + Symbol('10110') - \
     Symbol('11001') + Symbol('11010') - Symbol('11100') + Symbol('11111')

We will now define the Pauli matrices that will be used later on to construct the different discrete errors.

In [4]:
X = Matrix([[0, 1], [1, 0]])
Y = Matrix([[0, -I], [I, 0]])
Z = Matrix([[1, 0], [0, -1]])

The computational basis ($B_c$) can be defined as follows. We will also define a method to build any binary string, from the non-negative integer in decimal basis that it should represent.

In [5]:
def convert_to_binary_string(i: int, num_qubits: int) -> Symbol:
    """
    This method returns the given non-negative integer in decimal basis, expressed as
    a binary string with as many digits as required by the second parameter.

    :param i: Non-negative integer in decimal basis.
    :param num_bits: Length of the resulting binary string.
    :return: Sympy symbol containing the desired string as name.
    """
    binary_string = str(bin(i)[2:])
    binary_string = ''.join(['0' for _ in range(num_qubits - len(binary_string))]) + binary_string
    return Symbol(binary_string)


B_c = []  # Computational basis: [ 00000, 00001, ..., 11111 ].
for i in range(2 ** num_qubits):
    B_c.append(convert_to_binary_string(i, num_qubits))
print('B_c:', B_c)

B_c: [00000, 00001, 00010, 00011, 00100, 00101, 00110, 00111, 01000, 01001, 01010, 01011, 01100, 01101, 01110, 01111, 10000, 10001, 10010, 10011, 10100, 10101, 10110, 10111, 11000, 11001, 11010, 11011, 11100, 11101, 11110, 11111]


The basis associated with the quantum code $C$ ($B$) can be defined as follows, using the Pauli matrices, and aplying them on all the different qubits of each of the quantum states defined as generators, to obtain:
$B = [|0_L\rangle, |1_L\rangle, X_0|0_L\rangle, X_0|1_L\rangle, X_1|0_L\rangle, X_1|1_L\rangle, ..., Z_4|0_L\rangle, Z_4|1_L\rangle]$

We need to define an auxiliary method to apply 1-qubit operation to quantum states, using its string representation to perform the calculation. We will also need another method to obtain which new states could be obtained if we alter one bit of an incoming binary string.

In [6]:
def apply_1qubit_operation(operation: Matrix, target_qubit: int, initial_state: Expr, num_qubits: int) -> Expr:
    """
    This method applies the specified 1-qubit operation to the initial state, on the desired
    target qubit, and returns the resulting quantum state.
    
    :param operation: 2x2 matrix to be applied. 
    :param target_qubit: Qubit on which it should be applied, one of {0, 1, ..., num_qubits - 1}.
    :param initial_state: Initial quantum state, as an expression in terms of elements of the
        computational basis, expressed as binary strings.
    :param num_qubits: Number of qubits of the quantum state.
    :return: New quantum state after applying the operation.
    """
    
    final_state = 0

    for i in range(2 ** num_qubits):
        index = convert_to_binary_string(i, num_qubits)

        alpha = initial_state.coeff(index)
        state_0, state_1 = get_basis_states_one_bit_apart(index, target_qubit)

        initial_qubit_state = Matrix([[1], [0]]) if index.name[target_qubit] == '0' else Matrix([[0], [1]])
        final_qubit_state = operation * initial_qubit_state

        final_state += alpha * (final_qubit_state[0] * state_0 + final_qubit_state[1] * state_1)

    return final_state

def get_basis_states_one_bit_apart(reference_state: Symbol, target: int) -> tuple:
    """
    Method used to get the two binary strings that would be obtained if we modify any
    one bit of an incoming binary string. E.g.: if we receive the string 00000 and the
    integer '1' as target, the two possible outputs will be 00000 and 01000.
    
    :param reference_state: Incoming binary string. 
    :param target: Bit that would be altered, one of {0, 1, ..., length - 1}.
    :return: Tuple with the new two binary strings.
    """
    reference_state = reference_state.name

    return Symbol(reference_state[:target] + '0' + reference_state[target + 1:]), \
           Symbol(reference_state[:target] + '1' + reference_state[target + 1:])

B = [L0, L1]  # Basis associated with the code C.
for operator in [X, Y, Z]:
    for u in range(num_qubits):
        for generator in [L0, L1]:
            B.append(
                apply_1qubit_operation(operator, u, generator, num_qubits)
            )
print('B:', B)

B: [00000 - 00011 + 00101 - 00110 + 01001 + 01010 - 01100 - 01111 - 10001 + 10010 + 10100 - 10111 - 11000 - 11011 - 11101 - 11110, -00001 - 00010 - 00100 - 00111 - 01000 + 01011 + 01101 - 01110 - 10000 - 10011 + 10101 + 10110 - 11001 + 11010 - 11100 + 11111, -00001 + 00010 + 00100 - 00111 - 01000 - 01011 - 01101 - 01110 + 10000 - 10011 + 10101 - 10110 + 11001 + 11010 - 11100 - 11111, -00000 - 00011 + 00101 + 00110 - 01001 + 01010 - 01100 + 01111 - 10001 - 10010 - 10100 - 10111 - 11000 + 11011 + 11101 - 11110, 00001 + 00010 - 00100 - 00111 + 01000 - 01011 + 01101 - 01110 - 10000 - 10011 - 10101 - 10110 - 11001 + 11010 + 11100 - 11111, -00000 + 00011 + 00101 - 00110 - 01001 - 01010 - 01100 - 01111 - 10001 + 10010 - 10100 + 10111 - 11000 - 11011 + 11101 + 11110, 00001 - 00010 + 00100 - 00111 - 01000 - 01011 + 01101 + 01110 + 10000 - 10011 - 10101 + 10110 - 11001 - 11010 - 11100 - 11111, -00000 - 00011 - 00101 - 00110 + 01001 - 01010 - 01100 + 01111 + 10001 + 10010 - 10100 - 10111 - 11000 

## 2. Finding the change of basis matrices

Now we will define the matrix $M$ used to transform from basis $B$ to basis $B_c$. We will define it in two ways, firstly by putting the vectors from $B$, expressed in tearms of $B_c$, as columns. Then, we will define it by using derivatives. After that, we will ensure that we can get all vectors of the base $B$ expressed in the computational basis $B_c$ and that matches the expression of the basis $B$ obtained earlier, and also that both matrices $M$ are equal, as confirmations.

In [7]:
M = Matrix([
    [
        B[column].coeff(convert_to_binary_string(row, num_qubits))
        for column in range(2 ** num_qubits)
    ] for row in range(2 ** num_qubits)
])

In [8]:
M2 = Matrix([
    [
        diff(B[j], B_c[i]) for j in range(2 ** num_qubits)
    ] for i in range(2 ** num_qubits)
])

In [9]:
def get_computational_basis_vector_state(i: int, num_qubits: int) -> Matrix:
    """
    This methods returns a column vector that expresses the quantum state from the
    computational basis expressed by the integer 'i', for a quantum state with a total
    of 'num_qubits' qubits.

    :param i: Non-negative integer in decimal basis.
    :param num_qubits: Number of qubits of the resulting quantum state.
    :return: Sympy matrix containing the quantum state as a column vector.
    """
    return Matrix([
        [
            1 if i == j else 0
        ] for j in range(2 ** num_qubits)
    ])

def transform_vector_state_to_expression_state(state: Matrix, num_qubits: int) -> Expr:
    """
    This method converts a column vector to an expression in terms of the states of
    the corresponding basis, as binary strings, to be used in methods that use these
    binary strings to perform calculations.
    
    :param state: Matrix containing a quantum state expressed as a column vector.
    :param num_qubits: Number of qubits of the quantum state.
    :return: Quantum state, as expression with its variables as binary strings.
    """

    result = 0

    for j in range(state.shape[0]):
        binary_string = convert_to_binary_string(j, num_qubits)
        result += state[j, 0] * binary_string

    return result

assert 0 == sum(M - M2)
for i in range(2 ** num_qubits):
    state_B_vector = M * get_computational_basis_vector_state(i, num_qubits)
    state_B_c_expression = transform_vector_state_to_expression_state(state_B_vector, num_qubits)
    assert state_B_c_expression == B[i]

print('Shape of M == M2:', M.shape)
#pprint(M)

Shape of M == M2: (32, 32)


## 3. Finding the unitary error operators

Now we will define the qubit errors $W_u$ in terms of the coefficients $A_u$, $B_u$, $C_u$ and $D_u$.

In [10]:
A = symbols('A_0:%d' % num_qubits)  # Define A_0 through A_4.
B = symbols('B_0:%d' % num_qubits)  # Define A_0 through A_4.
C = symbols('C_0:%d' % num_qubits)  # Define A_0 through A_4.
D = symbols('D_0:%d' % num_qubits)  # Define A_0 through A_4.

W_u = []
for u in range(num_qubits):  # Define W_0 through W_4.
    W_u.append(Matrix([
        [A[u] + I * B[u], -C[u] + I * D[u]],
        [C[u] + I * D[u], A[u] - I * B[u]]
    ]))
    print('W[%d]:' % u, W_u[u])

W[0]: Matrix([[A_0 + I*B_0, -C_0 + I*D_0], [C_0 + I*D_0, A_0 - I*B_0]])
W[1]: Matrix([[A_1 + I*B_1, -C_1 + I*D_1], [C_1 + I*D_1, A_1 - I*B_1]])
W[2]: Matrix([[A_2 + I*B_2, -C_2 + I*D_2], [C_2 + I*D_2, A_2 - I*B_2]])
W[3]: Matrix([[A_3 + I*B_3, -C_3 + I*D_3], [C_3 + I*D_3, A_3 - I*B_3]])
W[4]: Matrix([[A_4 + I*B_4, -C_4 + I*D_4], [C_4 + I*D_4, A_4 - I*B_4]])


With this, we can express the unitary error operator for independent qubit errors $W$, and that same operator in the base $B$, $W_B$:

In [11]:
W = TensorProduct(W_u[0], W_u[1], W_u[2], W_u[3], W_u[4])
print('Shape of W:', W.shape)
#pprint(W)

Shape of W: (32, 32)


In [12]:
W_B = Dagger(M) * W * M
print('Shape of W_B:', W.shape)
#pprint(W_B)

Shape of W_B: (32, 32)


## 4. Disturbing a quantum state

We can define a generic quantum state in basis $B$, $\phi_B$ (phi_B), as follows:

In [13]:
w = symbols('w_0:%d' % 4)  # Define w_0 through w_3.

phi_B = Matrix([
    [w[0] + I * w[1]],
    [w[2] + I * w[3]]
] + [
    [0]
    for _ in range(2 ** num_qubits - 2)
])
print('Shape of phi_B:', phi_B.shape)
print(phi_B)

Shape of phi_B: (32, 1)
Matrix([[w_0 + I*w_1], [w_2 + I*w_3], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])


The disturbed quantum state $\psi_B$ (psi_B) is obtained by multiplying $\phi_B$ by the error operator $W_B$:

In [14]:
psi_B = W_B * phi_B
print('Shape of psi_B:', psi_B.shape)
#print(psi_B)

Shape of psi_B: (32, 1)


### 4.1 Proving Lemma 1

Lemma 1 states that the coordinates of the disturbed state $\psi_B = (\beta_0, \beta_1, ..., \beta_{31})_B$ satisfy:

$$\beta_{2s} = (a_s + ib_s)w_0 + (-b_s+ia_s)w_1 + (c_s + id_s)w_2 + (-d_s + ic_s)w_3$$
$$\beta_{2s+1} = (c_s - id_s)w_0 + (d_s+ic_s)w_1 + (-a_s + ib_s)w_2 + (-b_s - ia_s)w_3$$
$$0 \le s < 16$$

In [15]:
# ----------------------------------------------
#         WORK IN PROGRESS
# ----------------------------------------------

a = []
b = []
c = []
d = []
for s in range(16):
    print('s:', s, '\n')

    beta_2s_1000 = psi_B[0].subs(w[0], 1).subs(w[1], 0).subs(w[2], 0).subs(w[3], 0)
    a.append(re(beta_2s_1000))
    b.append(im(beta_2s_1000))
    print('a_%d:' % s, a[s], '\n')
    print('b_%d:' % s, b[s], '\n')

    beta_2s_0010 = psi_B[0].subs(w[0], 0).subs(w[1], 0).subs(w[2], 1).subs(w[3], 0)
    c.append(re(beta_2s_0010))
    d.append(im(beta_2s_0010))
    print('c_%d:' % s, c[s], '\n')
    print('d_%d:' % s, d[s], '\n')

    beta_2s = (a[s] + I*b[s]) * w[0] + (-b[s] + I*a[s]) * w[1] + (c[s] + I*d[s]) * w[2] - (-d[s] + I*c[s]) * w[3]
    print(beta_2s, '\n')
    print(psi_B[2 * s], '\n')
    assert beta_2s == psi_B[2 * s, 0]

    beta_2s_1 = (c[s] - I*d[s]) * w[0] + (d[s] + I*c[s]) * w[1] + (-a[s] + I*b[s]) * w[2] - (-b[s] - I*a[s]) * w[3]
    print(beta_2s_1, '\n')
    print(psi_B[2 * s + 1], '\n')
    assert beta_2s_1 == psi_B[2 * s + 1, 0]

s: 0 

a_0: 16*re(A_0*A_1*A_2*A_3*A_4) + 16*re(A_0*B_1*B_4*C_2*C_3) + 16*re(A_0*B_2*B_3*D_1*D_4) + 16*re(A_0*C_1*C_4*D_2*D_3) + 16*re(A_1*B_0*B_2*C_3*C_4) + 16*re(A_1*B_3*B_4*D_0*D_2) + 16*re(A_1*C_0*C_2*D_3*D_4) + 16*re(A_2*B_0*B_4*D_1*D_3) + 16*re(A_2*B_1*B_3*C_0*C_4) + 16*re(A_2*C_1*C_3*D_0*D_4) + 16*re(A_3*B_0*B_1*D_2*D_4) + 16*re(A_3*B_2*B_4*C_0*C_1) + 16*re(A_3*C_2*C_4*D_0*D_1) + 16*re(A_4*B_0*B_3*C_1*C_2) + 16*re(A_4*B_1*B_2*D_0*D_3) + 16*re(A_4*C_0*C_3*D_1*D_2) - 16*im(A_0*A_1*B_3*C_2*C_4) - 16*im(A_0*A_2*B_1*D_3*D_4) - 16*im(A_0*A_3*B_4*D_1*D_2) - 16*im(A_0*A_4*B_2*C_1*C_3) - 16*im(A_1*A_2*B_4*C_0*C_3) - 16*im(A_1*A_3*B_2*D_0*D_4) - 16*im(A_1*A_4*B_0*D_2*D_3) - 16*im(A_2*A_3*B_0*C_1*C_4) - 16*im(A_2*A_4*B_3*D_0*D_1) - 16*im(A_3*A_4*B_1*C_0*C_2) - 16*im(B_0*B_1*B_2*B_3*B_4) - 16*im(B_0*C_2*C_3*D_1*D_4) - 16*im(B_1*C_3*C_4*D_0*D_2) - 16*im(B_2*C_0*C_4*D_1*D_3) - 16*im(B_3*C_0*C_1*D_2*D_4) - 16*im(B_4*C_1*C_2*D_0*D_3) 

b_0: 16*re(A_0*A_1*B_3*C_2*C_4) + 16*re(A_0*A_2*B_1*D_3*D_4)

(w_0 + I*w_1)*((A_0 - I*B_0)*(A_1 - I*B_1)*(A_2 - I*B_2)*(A_3 - I*B_3)*(A_4 + I*B_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_2 - I*B_2)*(A_3 + I*B_3)*(A_4 - I*B_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_2 - I*B_2)*(-C_3 + I*D_3)*(C_4 + I*D_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_2 - I*B_2)*(C_3 + I*D_3)*(-C_4 + I*D_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_2 + I*B_2)*(A_3 - I*B_3)*(A_4 - I*B_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_2 + I*B_2)*(A_3 + I*B_3)*(A_4 + I*B_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_2 + I*B_2)*(-C_3 + I*D_3)*(-C_4 + I*D_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_2 + I*B_2)*(C_3 + I*D_3)*(C_4 + I*D_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_3 - I*B_3)*(-C_2 + I*D_2)*(C_4 + I*D_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_3 - I*B_3)*(C_2 + I*D_2)*(-C_4 + I*D_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_3 + I*B_3)*(-C_2 + I*D_2)*(-C_4 + I*D_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_3 + I*B_3)*(C_2 + I*D_2)*(C_4 + I*D_4) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_4 - I*B_4)*(-C_2 + I*D_2)*(C_3 + I*D_3) + (A_0 - I*B_0)*(A_1 - I*B_1)*(A_4 - I*B_4

AssertionError: 